# 2. Features and Vectors

## 2.1 Extraction and Vectorization

In stylometry, feature vectors are numerical representations of text segments and their lexical composition.  Hence, vectors guarantee an objective basis for comparability of texts. The power of this, is that algorithms can process textual characteristics automatically, simultaneously, fast and on a large scale, often disregarding word order (the **bag-of-words** approach). A vector α, filled with the feature frequencies of text α, after all, becomes a solid basis for comparison with vectors β, γ, ... which in their own turn represent the stylistic properties of text samples β, γ, ... This common ground allows to approach stylistic difference literally as **mathematical difference** and (as we will see when visualizing the data) **geometric distance**.

Before we get started, first run the code below again (copy of preprocessing code from the previous notebook) in order to make our text data manipulable.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import re
import glob
from string import punctuation

# Declare empty lists to fill up with our metadata and data
authors, titles, texts = [], [], []

# We declare some parameters — the 'settings' of our stylometric experiments
sample_len = 1400 # word length of text segment

# Function to clean and split text
def clean_and_split_text(text, sample_len):
    words = re.sub(r'[\d%s]' % re.escape(punctuation), '', text.lower()).split()
    return [words[i:i + sample_len] for i in range(0, len(words), sample_len)]

for filename in uploaded.keys():
    author, title = filename.split('/')[-1].split('.')[0].split('_')[:2]
    with open(filename, encoding='utf-8-sig') as file:
        text = file.read().strip()
        bulk = clean_and_split_text(text, sample_len)

        for index, sample in enumerate(bulk):
            if len(sample) == sample_len:
                authors.append(author)
                titles.append(f"{title}_{index + 1}")
                texts.append(" ".join(sample))

# Print summary to confirm things worked
print("Text processing complete!")
print(f"Number of text segments: {len(texts)}")
print(f"Number of authors: {len(set(authors))}")
print(f"Number of titles/segments: {len(titles)}")

# Optional: print a sample segment
print("\nSample processed text segment:\n", texts[0][:200], "...")

The **[scikit-learn](https://scikit-learn.org/stable/index.html) "Machine Learning in Python"-library** offers tremendously handy modules and transformers to extract features in a format supported by machine learning algorithms.

Below, we initialize our first vectorizer, under the variable `model`, which helps extract the textual features we are interested in (`word`, `char`, meaning respectively 'word' or 'character'), the number of features (`max_features=20`), but which can also be helpful to eliminate words that are irrelevant to our focus (`stop_words`), or where we can feed a restrictive list of words in advance that narrows the selection down (`vocabulary`) would be a comprehensive list containing those features which you want to look at exclusively). The `n_gram` range, consequently, formulates the sequences of *n* characters / words / parts-of-speech one wants to look at.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize sklearn vectorizer model
model = CountVectorizer(stop_words=[], # filter words
                        max_features=20, # n features = vector length / vector dimensionality.
                        analyzer='word', # feature type
                        vocabulary=None, # takes list of fixed words
                        ngram_range=((1,1)))

X = model.fit_transform(texts).toarray()

# Print confirmation
print("Vectorization complete!")
print(f"Number of text segments: {X.shape[0]}")
print(f"Vector dimensionality / features: {X.shape[1]}")
print("\nFeature names:", model.get_feature_names_out())

print(X)

The result of this process, the matrix `X` printed above, is our very first matrix containing vectors representing the feature frequencies of each of our text segments. The `toarray()` converts X into a standard array format, typically `numpy`-array.  
In `X`, each row equals a text segment. Every column stands for the frequency of an extracted feature.

In the code block below, we make the above more intuitive by representing `X`in a so-called DataFrame (abbreviated as `df`) using `pandas`. This is an open-source data analysis and manipulation library in Python that provides data structures and functions needed to manipulate structured data.

In [ ]:
import pandas as pd

df = pd.DataFrame(X) # structures matrix X as a DataFrame
features = model.get_feature_names_out()

# Assigns labels to colu`mns and rows (i.e. features and title segments)
df.columns = features # assigns column labels
df.index = [title for title in titles] # assigns row labels

print(df)

At this point, the scikit-learn feature extraction model still sorts the columns by alphabet rather than by frequency. The code below shows you how to refit your vectorizer so the vectors, somewhat more intuitively, descend from higher to lower frequency.

In [ ]:
import numpy as np

# Sum the feat frequencies across all documents (i.e. text segments)
feat_frequencies = np.asarray(X.sum(axis=0)).flatten()

# Get the feature names (features)
features = model.get_feature_names_out()

# Create a DataFrame with features and their frequencies
feat_freq_df = pd.DataFrame({'feature': features, 'frequency': feat_frequencies})

# Sort the DataFrame by frequency in descending order
feat_freq_df = feat_freq_df.sort_values(by='frequency', ascending=False).reset_index(drop=True)

# Get the sorted features
sorted_features = feat_freq_df['feature'].tolist()

# Reorder the columns of the matrix X according to the sorted features
sorted_indices = [model.vocabulary_[feat] for feat in sorted_features]
X_sorted = X[:, sorted_indices]

# Print the sorted feature names and their corresponding frequencies
print(feat_freq_df)
print(sorted_features)

Now you can refit a new `CountVectorizer` on our texts by feeding the `sorted_features`container to the `vocab` parameter of the vectorizer.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize sklearn model
model = CountVectorizer(stop_words=[],
                        max_features=20,
                        analyzer='word',
                        vocabulary=sorted_features,
                        ngram_range=((1,1)))

X = model.fit_transform(texts).toarray()

df = pd.DataFrame(X)
features = model.get_feature_names_out()

# Assigns names to columns and features (i.e. features and title segments)
df.columns = features
df.index = [title for title in titles]

print(df)

Try to work with different settings, and then look at the various results in the extraction and vectorization process.

Below we introduce another common type of vectorization called **TF-IDF-vectorization**, which stands for ‘term frequency-inverse document frequency’, see C. D. Manning, P. Raghavan, and H. Schütze, *Introduction to Information Retrieval* (New York, Cambridge University Press, 2008), at 289-290. It divides all feature values by the number of documents that respective feature appears in. As a consequence, less common features receive a higher weight, which prevents them from sinking away (and from losing statistical significance) amidst more common features. You can try to fit a TfidfVectorizer to your own dataset by running the code block below.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize sklearn model
model = TfidfVectorizer(stop_words=[],
                        max_features=20,
                        analyzer='word',
                        vocabulary=sorted_features,
                        ngram_range=((1,1)))

X = model.fit_transform(texts).toarray()

df = pd.DataFrame(X)
features = model.get_feature_names_out()

# Assigns names to columns and features (i.e. features and title segments)
df.columns = features
df.index = [title for title in titles]

print(df)

## 2.2 Distance Measures and Scaling

### 2.2.1 Understanding Distance: Burrows' Delta

Popularly applied to look at distances between vectors in the recent history of computational stylistics, are **John Burrows’** distance measures **Delta** (2002), **Zeta** and **Iota**. Burrows' methods, especially Delta, proved invaluable in the development of stylometry in its own time, and continues to provide an intuitive introduction to stylometric research. Due to the proliferation and improved understanding of the mathematical-statistical precepts underlying his theory, applying Delta by itself is not often carried out anymore and has become a tad outdated. We will explain why below.

With Delta, Burrows essentially operationalized and matched two data-analytical steps: he first **'normalized'** or **'scaled'** the frequencies of words, and afterward calculated a **distance** between these vectors. The scaling implied that Burrows ‘standardized’ the word frequencies over the whole corpus so that the mean for each word is 0, and the standard deviation is 1, also known as the ‘z-score’. Consequently, he applied the so-called 'Manhattan' distance, to be treated further below.

It turns out that Burrows' way of normalizing and measuring distance was, however, just one of many ways in which to conceptualize the distance between vectors. Argamon (2008) showed that, despite it being very effective, it may even not necessarily be the most optimal way. There have been several proposals to improve Delta (Hoover 2004b, Argamon 2008, Eder, Smith and Aldridge 2011, Kestemont and Rybicki 2013).

If we want to implement Burrows' Delta in Python, we have to normalize our text vectors in `X` by transforming them to z scores, which can easily be done by importing `StandardScaler()` from `sklearn`. Consequently, we can apply the distance metric **Manhattan** (`sklearn.metrics.pairwise`) to calculate the similarity between the data points of a test text and a target corpus. Note that we should not necessarily opt for Manhattan distance. Other distance metrics, especially **Euclidean** and **Cosine**, provide valuable routes that can (and in fact should) be tested in performance. There are quite a few distance measures (or distance functions) around, but arguably they are are all variations falling under one of these three commonly encountered measures.

* **Manhattan** or ‘taxicab distance’ has its onomasty in a city street plan, is a distance function which measures two points along axes at right angles, indeed, as if one would be navigating in a city along city blocks.
* **Euclidean** distance is commonly the most straightforward way of going from one place to another, and can be generally thought of as the distance in which ‘the crow flies,’ directly fixing one point to another without taking any turns.
* **Cosine** instead measures the cosine of the angle between the points.

### 2.2.2 Scaling

More often than not, analysis will be carried out not on raw frequencies of features, but on ‘normalized’ or ‘weighted’ frequencies (feature scaling). In fact, TF-IDF is arguably already a (be it subtle) way of scaling feature frequencies.

Scaling data is crucial in all statistical methods. It transforms the data to ensure that different features are on a similar scale. Key advantages are improved performance of data-analytical operations at a later stage, the equal importance (weight) granted by each of the respective features in the dataset, and a more intuitive interpretability of the results.

With `sklearn.preprocessing` it is possible to import scaling models such as `Normalizer`, `StandardScaler`, `MinMaxScaler`, and many others, to apply scaling. The block of code below is an illustration.

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

np.set_printoptions(suppress=True, precision=2) # suppresses scientific notation

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(X_scaled)

Once we have scaled vectors, various distance measures can be applied. The block of code below yields a matrix  of pairwise differences (with a dimension or `np.shape` of 250x250) between the text segment and all text segments in the corpus, including... itself! Obviously, the distance is 0 in that case.

Have a go and run the block of code below. For the first time, we will also use a plot (a heatmap) to visualize what is going on, making use of the Python library `matplotlib`. The heatmap visualizes the Manhattan distances between our text segment vectors. Each cell in the heatmap represents the distance between two vectors, with the color intensity indicating the magnitude of the distance. Darker colors correspond to smaller distances (i.e., more similar vectors), while lighter colors indicate larger distances (i.e., less similar vectors), see the color bar on the right provides. Both the x- and y-axes represent the indices of the vectors, making it possible to identify which vectors are being compared in each cell.

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances
import matplotlib.pyplot as plt
from datetime import datetime

distance_matrix = manhattan_distances(X_scaled)
print(distance_matrix)

# Plot a heatmap
fig = plt.figure()
plt.imshow(distance_matrix, cmap='hot', interpolation='nearest')
plt.colorbar(label='Distance')
plt.title('Heatmap of Vector Distances')
plt.xlabel('Vector Index')
plt.ylabel('Vector Index')
plt.show()

## 2.3 Feature Weighting and Feature Selection

Not to be confused with feature extraction, **feature selection** techniques penalize / filter out those features that are considered redundant, and reward or include only the best discriminants.

Sometimes, the selection or elimination of features is done manually on the basis of **qualitative** arguments (Kestemont et al. 2015:206). However, most feature selection base themselves on **quantitative** criteria (where a distinction is made between filter-based and wrapper-based methods).

The classes in the `sklearn.feature_selection` are useful for feature selection/dimensionality reduction to improve estimators’ accuracy scores or to boost their performance on very high-dimensional datasets. Machine-learning techniques, to be treated later today, can be particularly helpful in evaluating the efficiency and relevance of including or excluding certain features. This relevance is often determined by **weights**: establishing feature weights is often a first step toward a pruned and more efficient set of discriminants.

The idea that some words deserve less or more attention (again, deserve more **weight**) than others in attribution disputes is an idea that arose fairly early in stylometric research. Proposed by Hoover (2004), for instance, was '**culling**', where features that do not appear in all the representative texts of a corpus have to be disregarded.

### 2.3.1 An Intuition through Burrows' and Craig's Zeta

Above, we already mentioned Burrows's '**Zeta**' in passing. Zeta was first proposed in 2007 and modified by other scholars (Argamon 2008, Craig and Kinney 2009, Hoover 2004a, 2004b). In what follows, we will explain Zeta just as one of many possible ways to think about feature selection. Two main reasons: (1) Zeta is a commonly encountered state-of-the-art technique stylometric research, and (2) unlike other feature selection methods, there is less available Python code lying around for it in libraries or modules.

John Burrows motivated the advantage of Zeta mainly from the idea that the frequency of a feature by itself can be misleading when talking about its significance for the author in question. It is, for instance, very possible that a certain word in a text occurs many times in the author's Prologue, but then again, for instance, fails to turn up later in the text. what we also need to take into account, he argued, is the **dispersion** of the word over a text or entire oeuvre. Instead of look at the frequencies of words, we concentrate on their consistency of appearance.

Some key ideas to take home:
* The original Zeta compared subcorpora by two and not more authors, **Author A** and **Author B** below, and we will do the same here.
* Zeta is about frequency per segment (**proportion** or **ratio**) rather than raw, total word count.
* Zeta is mainly about 'distinctiveness', i.e. **preference and avoidance** by Authors A or B: Zeta returns a list of words that are statistically either preferred or avoided in each subcorpus.
* As a consequence, we see that Zeta analysis often excludes the extremely common words that are traditionally the focus of stylometry, and concentrates on the **middle of the word frequency spectrum**, yielding a few extremely efficient features for making distinction between Author A and Author B.
* Sample or **segment length** is an important parameter in its performance.

As an additional parameter, you can also set a threshold: e.g. *w* needs to occur at least once per segment for one of both authors under consideration, before we consider it to have importance.

Zeta combines the ratio of the sections by one author in which each word occurs with the ratio of the sections by the other author into a single measure of 'distinctiveness' for each word. It does so by subtracting the zeta scores of Author B from that of Author A. This 'composite score' produces two lists of words: one favored by the first author and avoided by the second, the other favored by the second author and avoided by the first.

Concretely, this means:
* words that score `0` are used by exactly the same frequency per segment by both Authors A and B.
* words that score `-1` are used by Author A in every segment, and not in any segment by Author B.
* words that score `1` are used by Author B in every segment, and not in any segment by Author A.

The code block below simply rehearses a lot we have seen earlier, only this time, we **vectorize the entire vocabulary** in a long feature vector, instead of only looking at the most common words.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
import numpy as np
import pandas as pd
import re

# Declare empty lists to fill up with our metadata and data
authors = []
titles = []
texts = []

# We declare some parameters — the 'settings' of our stylometric experiments
sample_len = 1400 # word length of text segment

# Open all file objects in folder and gather data
for filename in uploaded.keys():
    author = filename.split("/")[-1].split(".")[0].split("_")[0]
    title = filename.split("/")[-1].split(".")[0].split("_")[1]

    bulk = []
    text = open(filename, encoding='utf-8-sig').read() # utf-8-sig encoding automatically handles and removes Unicode Byte Order Mark (BOM) if present

    # .split() method splits string into list of substrings based on a specified delimiter. By default, the delimiter is a whitespace
    # .strip() method removes leading and trailing whitespace from a string: spaces, tabs, newlines, and other whitespace characters.
    for word in text.strip().split():
        word = re.sub(r'\d+', '', word) # escapes digits
        word = re.sub('[%s]' % re.escape(punctuation), '', word) # escape punctuation
        word = word.lower() # convert upper to lowercase
        bulk.append(word)

    # Split up the text into discrete chunks or segments
    bulk = [word for word in bulk if word != ""] # list comprehension that removes emptry strings
    bulk = [bulk[i:i+sample_len] for i in range(0, len(bulk), sample_len)]
    for index, sample in enumerate(bulk):
        if len(sample) == sample_len:
            authors.append(author)
            titles.append(title + "_{}".format(str(index + 1)))
            texts.append(" ".join(sample))

# In this vectorizer, for the purpose of Zeta, no max_features is given.
model = CountVectorizer(stop_words=[], # filter words
                        analyzer='word', # feature type
                        vocabulary=None, # takes list of fixed words
                        ngram_range=((1,1)))

X = model.fit_transform(texts).toarray()
features = model.get_feature_names_out()

print(features)
print(X.shape) # prints dimension (n_samples, n_feats)

In the code block below, we first have to indicate which authors we are interested in by typing out the author names (corresponding to the file names) in the list variable `authors_of_interest`.

Consequently, we loop over matrix `X`, taking into account only the text segment vectors of interest, and crunch them into `bit_vectors`, or 'binarize them', in the sense that any frequency greater than `0` is simply converted into `1`.

We can consequently count (as we do in the line of code containing `counts = np.sum(bit_vector)`) how many times the word occurs more than 0 in each text segment. If we divide this number by the variable `n_segments`, we get a `ratio`.


In [ ]:
# Select the two authors we want to compare
authors_of_interest = ['Hildegardis-Bingensis', 'Bernardus-Claraevallensis']

# Initialize a dictionary to store bit vectors for each author
new_X = {author: [] for author in authors_of_interest}

# Convert feature counts to binary presence/absence for the selected authors
for author, title, x_vec in zip(authors, titles, X):
    if author in authors_of_interest:
        x_vec[x_vec > 0] = 1  # convert counts > 0 to 1 (presence/absence)
        new_X[author].append(x_vec)

# Convert lists of vectors to numpy arrays for easier computation
new_X = {author: np.array(bit_matrix) for author, bit_matrix in new_X.items()}

# Compute per-word ratios: in how many segments each word appears for each author
ratios_dict = {author: None for author in authors_of_interest}
for author, bit_matrix in new_X.items():
    n_segments = bit_matrix.shape[0]  # number of text segments for this author
    all_ratios = []
    for feat, bit_vector in zip(features, bit_matrix.transpose()):  # transpose: iterate over words
        counts = np.sum(bit_vector)  # number of segments in which the word occurs
        ratio = counts / n_segments   # ratio of segments with the word
        all_ratios.append(ratio)
    ratios_dict[author] = all_ratios  # store ratios per author

# Prepare matrix for Zeta calculation
# negative -1: preference by Author A
# 0: same usage by both authors
# positive +1: preference by Author B
X_ratios = [all_ratios for all_ratios in ratios_dict.values()]
X_ratios = np.array(X_ratios).transpose()
y = [author for author in ratios_dict.keys()]

# Calculate Zeta: difference in segment ratios between authors
zeta_scores = X_ratios[:, 1] - X_ratios[:, 0]

# Sort words by Zeta scores
negatives = [(x, weight) for weight, x in sorted(zip(zeta_scores, features))]  # most negative = preferred by Author A
positives = negatives[::-1]  # most positive = preferred by Author B

# Show top 10 distinctive words for each author
print(negatives[:10])
print(positives[:10])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

# --- Build the word list as you had it ---
top_neg_words = [word for word, _ in negatives[:10]]
top_pos_words = [word for word, _ in positives[:10]]
top_words = top_neg_words + top_pos_words
word_indices = [list(features).index(w) for w in top_words]

# --- Distinctiveness (opacity) from |zeta| ---
# We normalize |zeta| across the selected top words to map into [min_alpha, 1.0]
zeta_for_top = np.abs(zeta_scores[word_indices])
zeta_max = zeta_for_top.max() if len(zeta_for_top) > 0 else 0.0
min_alpha = 0.15  # ensures low-distinctiveness words are still faintly visible

if zeta_max == 0:
    # If everything is equally non-distinctive, use a constant faint alpha
    alphas_per_word = np.full(len(word_indices), min_alpha, dtype=float)
else:
    alphas_per_word = min_alpha + (zeta_for_top / zeta_max) * (1.0 - min_alpha)

# --- Colors per author (colorblind-friendly) ---
palette = sns.color_palette("colorblind", n_colors=len(authors_of_interest))
author_colors = {a: palette[i] for i, a in enumerate(authors_of_interest)}

# --- Plot ---
fig, axes = plt.subplots(1, len(authors_of_interest), figsize=(16, 8), sharey=True)

n_words = len(top_words)
for i, author in enumerate(authors_of_interest):
    ax = axes[i] if isinstance(axes, np.ndarray) else axes

    # Get presence/absence for this author's segments on the selected words
    # new_X[author]: (n_segments, n_features)
    # We need (n_words, n_segments) for plotting with words on y-axis
    bit_matrix = new_X[author]
    sub = bit_matrix[:, word_indices].T.astype(float)  # shape: (n_words, n_segments)

    # --- Build RGBA image ---
    # base color is author-specific; alpha per cell = presence * alpha(word)
    r, g, b = author_colors[author]
    # broadcast RGB over the grid
    rgb = np.dstack([
        np.full_like(sub, r, dtype=float),
        np.full_like(sub, g, dtype=float),
        np.full_like(sub, b, dtype=float),
    ])
    # alpha per word, broadcast across segments, masked by presence (sub is 0/1)
    alpha = sub * alphas_per_word[:, None]
    rgba = np.dstack([rgb, alpha])

    im = ax.imshow(rgba, aspect="auto", interpolation="none", origin="upper")

    # Axes formatting
    ax.set_title(author)
    ax.set_xlabel("Segment")

    # Set y-ticks to the combined list of words
    axes[0].set_yticks(np.arange(len(top_words)) + 0.5)
    axes[0].set_yticklabels(top_words, rotation=0)

    # Keep words oriented top-to-bottom matching list order
    ax.set_ylim(n_words - 0.5, -0.5)

plt.suptitle("Word presence across segments — color by author, opacity by |zeta|")
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

As you can imagine, these words are powerfull stuff! They hand us quite effectively a number of discriminants that work very well in distinguishing two authors' lexical preferences from one another, and the usefulness of the discriminants is not confined by membership in the high frequency strata.

Below, we visualize our results by using `matplotlib` once more.

The variable `n_visualized` stands for 'number visualized': indicate how many of the top discriminants you wish to plot.

Consequently, take a good look at this plot, and try to verify if this makes sense to you on a qualitative level!

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt

# Parameters
n_visualized = 10 # number of terms based on Zeta score visualized

# Plot
fig, ax = plt.subplots()

# store Zeta data in variables
x_negatives = [tup[1] for tup in negatives[:n_visualized]]
x_positives = [tup[1] for tup in positives[:n_visualized]][::-1]
y = [i for i in range(0, n_visualized )]

# plot the values
for x_coord, y_coord in zip(y, x_positives):
    ax.barh(x_coord, y_coord, color='g', edgecolor='black', alpha=np.abs(y_coord))
for x_coord, y_coord in zip(y, x_negatives):
    ax.barh(x_coord, y_coord, color='r', edgecolor='black', alpha=np.abs(y_coord))

# labelling the axes
ax2 = ax.twinx()
ax2.set_ylim(ax.get_ylim())
bottom_labels = [tup[0] for tup in negatives[:n_visualized]]
top_labels = [tup[0] for tup in positives[:n_visualized][::-1]]
ax.set_yticks(y)
ax.set_yticklabels(bottom_labels)
ax2.set_yticks(y)
ax2.set_yticklabels(top_labels)

plt.title('Zeta scores')

# Despine both axes
for spine in ['right', 'top', 'left', 'bottom']:
    ax.spines[spine].set_visible(False)
    ax2.spines[spine].set_visible(False)

plt.axvline(x=0, lw=2, c='black', zorder=1)
